In [ ]:
# https://www.kaggle.com/shubh24/wordnet-similarity-matrix-a-naive-implementation

In [31]:
import pandas as pd
import numpy as np 
import numpy as np
import pandas as pd
import nltk
import re
import random
import collections
from nltk.corpus import stopwords
from nltk.corpus import wordnet as wn

from nltk.corpus import stopwords
stop = set(stopwords.words('english'))

df_train = pd.read_csv('data/train.csv')

In [35]:
def get_terms(text, lower=False, remove_not=False):
    lemmetizer = nltk.WordNetLemmatizer()

    # Clean the text
    text = re.sub(r"[^A-Za-z0-9]", " ", text)
    text = re.sub(r"what's", "", text)
    text = re.sub(r"What's", "", text)
    text = re.sub(r"\'s", " ", text)
    text = re.sub(r"\'ve", " have ", text)
    text = re.sub(r"can't", "cannot ", text)
    text = re.sub(r"n't", " not ", text)
    text = re.sub(r"I'm", "I am", text)
    text = re.sub(r" m ", " am ", text)
    text = re.sub(r"\'re", " are ", text)
    text = re.sub(r"\'d", " would ", text)
    text = re.sub(r"\'ll", " will ", text)
    text = re.sub(r"60k", " 60000 ", text)
    text = re.sub(r" e g ", " eg ", text)
    text = re.sub(r" b g ", " bg ", text)
    text = re.sub(r"\0s", "0", text)
    text = re.sub(r" 9 11 ", "911", text)
    text = re.sub(r"e-mail", "email", text)
    text = re.sub(r"\s{2,}", " ", text)
    text = re.sub(r"quikly", "quickly", text)
    text = re.sub(r" usa ", " America ", text)
    text = re.sub(r" USA ", " America ", text)
    text = re.sub(r" u s ", " America ", text)
    text = re.sub(r" uk ", " England ", text)
    text = re.sub(r" UK ", " England ", text)
    text = re.sub(r"india", "India", text)
    text = re.sub(r"switzerland", "Switzerland", text)
    text = re.sub(r"china", "China", text)
    text = re.sub(r"chinese", "Chinese", text) 
    text = re.sub(r"imrovement", "improvement", text)
    text = re.sub(r"intially", "initially", text)
    text = re.sub(r"quora", "Quora", text)
    text = re.sub(r" dms ", "direct messages ", text)  
    text = re.sub(r"demonitization", "demonetization", text) 
    text = re.sub(r"actived", "active", text)
    text = re.sub(r"kms", " kilometers ", text)
    text = re.sub(r"KMs", " kilometers ", text)
    text = re.sub(r" cs ", " computer science ", text) 
    text = re.sub(r" upvotes ", " up votes ", text)
    text = re.sub(r" iPhone ", " phone ", text)
    text = re.sub(r"\0rs ", " rs ", text) 
    text = re.sub(r"calender", "calendar", text)
    text = re.sub(r"ios", "operating system", text)
    text = re.sub(r"gps", "GPS", text)
    text = re.sub(r"gst", "GST", text)
    text = re.sub(r"programing", "programming", text)
    text = re.sub(r"bestfriend", "best friend", text)
    text = re.sub(r"dna", "DNA", text)
    text = re.sub(r"III", "3", text) 
    text = re.sub(r"the US", "America", text)
    text = re.sub(r"Astrology", "astrology", text)
    text = re.sub(r"Method", "method", text)
    text = re.sub(r"Find", "find", text) 
    text = re.sub(r"banglore", "Banglore", text)
    text = re.sub(r" J K ", " JK ", text)
    if lower == True:
        text = text.lower()
    
    words = nltk.word_tokenize(text)
    
    
    stops = set(stopwords.words("english"))
    not_stop = ['no', 'nor', 'not', "don't", "aren't", 'couldn', "couldn't", 'didn', "didn't", 'doesn', "doesn't",
                'hadn', "hadn't", 'hasn', "hasn't", 'haven', "haven't", 'isn', "isn't", 'mightn', "mightn't", 'mustn',
                "mustn't", 'needn', "needn't", 'shan', "shan't", 'shouldn', "shouldn't", 'wasn', "wasn't", 'weren',
                "weren't", 'won', "won't", 'wouldn', "wouldn't"]
    if remove_not == True:
        # stop = stops.difference(not_stop)
        stops = ['the', 'a', 'an', 'and', 'but', 'if', 'or', 'because', 'as', 'what', 'which', 'this', 'that', 'these',
                 'those', 'then',
                 'just', 'so', 'than', 'such', 'both', 'through', 'about', 'for', 'is', 'of', 'while', 'during', 'to',
                 'What', 'Which',
                 'Is', 'If', 'While', 'This']
    
    meaningful_words = [w for w in words if not w in stops]

    meaningful_words = [lemmetizer.lemmatize(w) for w in meaningful_words]
    
    return meaningful_words

In [41]:
# Pass a row and get pair score
def get_pair_score(row):
    res = row["is_duplicate"]
    terms1 = get_terms(row["question1"])
    terms2 = get_terms(row["question2"])

    sims = []
    
    for word1 in terms1:
        word1_sim = []

        try:
            syn1 = wn.synsets(word1)[0]
        except:  #if wordnet is not able to find a synset for word1
            sims.append([0 for i in range(0, len(terms2))])
            continue


        for word2 in terms2:
            try:
                syn2 = wn.synsets(word2)[0]
            except: #if wordnet is not able to find a synset for word2
                word1_sim.append(0)
                continue

            word_similarity = syn1.wup_similarity(syn2)
            word1_sim.append(word_similarity)

        sims.append(word1_sim)
        
        
    if(len(terms1)==0 or len(terms2)==0):
        return 0
        
    word1_score = 0
    for i in range(0, len(terms1), 1):
        try:
            word1_score += max(sims[i])
        except:
            continue
    word1_score /= len(terms1) #Averaging over all terms
    
    
    word2_score = 0
    for i in range(0, len(terms2), 1):
        try:
            word2_score += max([j[i] for j in sims])
        except:
            continue
    word2_score /= len(terms2)
    
    
    pair_score = (word1_score + word2_score)/2
    
    return pair_score

In [55]:
score_res = []

for j in range(50000):
    pair_score = get_pair_score(df_train.iloc[j])
    
    score_res.append([pair_score, df_train.iloc[j]['is_duplicate']])
    
    
score_res_df = pd.DataFrame(score_res, columns = ['pair_score', 'is_duplicate']) 
# score_res_df

In [56]:
duplicates = score_res_df.loc[score_res_df.is_duplicate==1]
nondupes = score_res_df.loc[score_res_df.is_duplicate==0]

median_nondup = np.median(nondupes['pair_score'])
median_nondup

0.24032538267832385

In [57]:
median_dup = np.median(duplicates['pair_score'])
median_dup

0.325

In [58]:
# Let's test
results_df = []

for j in range(50000,100000):
    pair_score = get_pair_score(df_train.iloc[j])
    decision = (median_nondup + median_dup)/2
    
    if(pair_score < decision):
        d_bin = 0
    else:
        d_bin = 1
        
    results_df.append([d_bin, df_train.iloc[j]['is_duplicate']])
                       

results_df = pd.DataFrame(results_df, columns = ['dup_bin', 'is_duplicate'])
                       

In [59]:
from sklearn.metrics import *
                                 
print("f1 " + str(f1_score(results_df['is_duplicate'], results_df['dup_bin'])))
print("accu " + str(accuracy_score(results_df['is_duplicate'], results_df['dup_bin'])))

f1 0.514053965351525
accu 0.58542


In [4]:
row = df_train.iloc[0] #Just taking the first row, you can put a loopover 

res = row["is_duplicate"]
terms1 = get_terms(row["question1"])
terms2 = get_terms(row["question2"])

sims = []

In [5]:
for word1 in terms1:
    word1_sim = []

    try:
        syn1 = wn.synsets(word1)[0]
    except:  #if wordnet is not able to find a synset for word1
        sims.append([0 for i in range(0, len(terms2))])
        continue


    for word2 in terms2:
        try:
            syn2 = wn.synsets(word2)[0]
        except: #if wordnet is not able to find a synset for word2
            word1_sim.append(0)
            continue

        word_similarity = syn1.wup_similarity(syn2)
        word1_sim.append(word_similarity)

    sims.append(word1_sim)

In [6]:
word1_score = 0
for i in range(0, len(terms1), 1):
    try:
        word1_score += max(sims[i])
    except:
        continue
word1_score /= len(terms1) #Averaging over all terms

In [7]:
word2_score = 0

for i in range(0, len(terms2), 1):
    try:
        word2_score += max([j[i] for j in sims])
    except:
        continue
word2_score /= len(terms2)

In [8]:
pair_score = (word1_score + word2_score)/2
pair_score

0.40476190476190477